# Market Basket Analysis using Instacart Online Grocery Dataset

## Which products will an Instacart consumer purchase again?

To showcase the concept of market basket analysis with the [Databricks Unified Analytics Platform](https://databricks.com/product/unified-analytics-platform), we will use the Instacart's [3 Million Instacart Orders, Open Sourced](https://www.instacart.com/datasets/grocery-shopping-2017) dataset.

> “The Instacart Online Grocery Shopping Dataset 2017”, Accessed from https://www.instacart.com/datasets/grocery-shopping-2017 on 01/17/2018. This anonymized dataset contains a sample of over 3 million grocery orders from more than 200,000 Instacart users.
For each user, we provide between 4 and 100 of their orders, with the sequence of products purchased in each order. We also provide the week and hour of day the order was placed, and a relative measure of time between orders.

Whether you shop from meticulously planned grocery lists or let whimsy guide your grazing, our unique food rituals define who we are. Instacart's grocery ordering and delivery app aims to make it easy to fill your refrigerator and pantry with your personal favorites and staples when you need them. After selecting products through the Instacart app, personal shoppers review your order and do the in-store shopping and delivery for you.


This notebook will show how you can predict which items a shopper will purchase whether they buy it again or recommend to try for the first time.
<img src="https://s3.us-east-2.amazonaws.com/databricks-dennylee/media/buy+it+again+or+recommend.png" width="1100"/>


*Source: [3 Million Instacart Orders, Open Sourced](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2)*

![](https://s3.us-east-2.amazonaws.com/databricks-dennylee/media/data-engineering-pipeline-3.png)

Data engineering pipelines are commonly comprised of these components:

* **Ingest Data**: Bringing in the data from your source systems; often involving ETL processes (though we will skip this step in this demo for brevity)
* **Explore Data**: Now that you have cleansed data, explore it so you can get some business insight
* **Train ML Model**: Execute FP-growth for frequent pattern mining
* **Review Association Rules**: Review the generated association rules

# Ingest Data

First, download the [3 Million Instacart Orders, Open Sourced](https://www.instacart.com/datasets/grocery-shopping-2017) and upload it to `dbfs`; for more information, refer to [Importing Data](https://docs.databricks.com/user-guide/importing-data.html).

The following `dbutils filesystem (fs)` query displays the six files:
* `Orders`: 3.4M rows, 206K users
* `Products`: 50K rows
* `Aisles`: 134 rows 
* `Departments`: 21 rows
* `order_products__SET`: 30M+ rows where `SET` is defined as:
  * `prior`: 3.2M previous orders
  * `train`: 131K orders for your training dataset
  
Reference: [The Instacart Online Grocery Shopping Dataset 2017 Data Descriptions](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b)

### Important
You will need to **edit** the locations (in the examples below, we're using `/mnt/bhavin/mba/instacard/csv`) to where you had uploaded your data.

# Importando as libs necessárias

In [1]:
import numpy as np
import pandas as pd
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [2]:
# Inicializa uma SparkSession
spark = SparkSession.builder \
    .appName("Aula1") \
    .getOrCreate()

# Subir os arquivos no cluster com a opção create table

In [4]:
# Import Data
aisles = spark.read.csv("../Aula1/Datasets/aisles.csv", header=True, inferSchema=True)
departments = spark.read.csv("../Aula1/Datasets/departments.csv", header=True, inferSchema=True)
order_products_prior = spark.read.csv("../Aula1/Datasets/order_products__prior.csv", header=True, inferSchema=True)
order_products_train = spark.read.csv("../Aula1/Datasets/order_products__train.csv", header=True, inferSchema=True)
orders = spark.read.csv("../Aula1/Datasets/orders.csv", header=True, inferSchema=True)
products = spark.read.csv("../Aula1/Datasets/products.csv", header=True, inferSchema=True)

# Create Temporary Tables
aisles.createOrReplaceTempView("aisles")
departments.createOrReplaceTempView("departments")
order_products_prior.createOrReplaceTempView("order_products_prior")
order_products_train.createOrReplaceTempView("order_products_train")
orders.createOrReplaceTempView("orders")
products.createOrReplaceTempView("products")

In [5]:
aisles.createOrReplaceTempView("aisles")
departments.createOrReplaceTempView("departments")
order_products_prior.createOrReplaceTempView("order_products_prior")
order_products_train.createOrReplaceTempView("order_products_train")
orders.createOrReplaceTempView("orders")
products.createOrReplaceTempView("products")

# Exploratory Data Analysis

Explore your Instacart data using Spark SQL

In [6]:
query = """
select 
  count(order_id) as total_orders, 
  (case 
     when order_dow = '0' then 'Sunday'
     when order_dow = '1' then 'Monday'
     when order_dow = '2' then 'Tuesday'
     when order_dow = '3' then 'Wednesday'
     when order_dow = '4' then 'Thursday'
     when order_dow = '5' then 'Friday'
     when order_dow = '6' then 'Saturday'              
  end) as day_of_week 
from orders  
group by order_dow 
order by total_orders desc
"""

result_dayweek = spark.sql(query)
result_dayweek.show()

+------------+-----------+
|total_orders|day_of_week|
+------------+-----------+
|      600905|     Sunday|
|      587478|     Monday|
|      467260|    Tuesday|
|      453368|     Friday|
|      448761|   Saturday|
|      436972|  Wednesday|
|      426339|   Thursday|
+------------+-----------+



In [29]:
query = """
select 
  count(order_id) as total_orders, 
  order_hour_of_day as hour 
  from orders 
 group by order_hour_of_day 
 order by order_hour_of_day
 """

result_hour = spark.sql(query)
result_hour.show()

+------------+----+
|total_orders|hour|
+------------+----+
|       22758|   0|
|       12398|   1|
|        7539|   2|
|        5474|   3|
|        5527|   4|
|        9569|   5|
|       30529|   6|
|       91868|   7|
|      178201|   8|
|      257812|   9|
|      288418|  10|
|      284728|  11|
|      272841|  12|
|      277999|  13|
|      283042|  14|
|      283639|  15|
|      272553|  16|
|      228795|  17|
|      182912|  18|
|      140569|  19|
+------------+----+
only showing top 20 rows



In [7]:
query = """
select countbydept.*
  from (
  -- from product table, let's count number of records per dept
  -- and then sort it by count (highest to lowest) 
  select department_id, count(1) as counter
    from products
   group by department_id
   order by counter asc 
  ) as maxcount
inner join (
  -- let's repeat the exercise, but this time let's join
  -- products and departments tables to get a full list of dept and 
  -- prod count
  select
    d.department_id,
    d.department,
    count(1) as products
    from departments d
      inner join products p
         on p.department_id = d.department_id
   group by d.department_id, d.department 
   order by products desc
  ) countbydept 
  -- combine the two queries's results by matching the product count
  on countbydept.products = maxcount.counter
"""

result_department_products = spark.sql(query)
result_department_products.show()

+-------------+---------------+--------+
|department_id|     department|products|
+-------------+---------------+--------+
|           16|     dairy eggs|    3449|
|           14|      breakfast|    1115|
|            3|         bakery|    1516|
|           21|        missing|    1258|
|            6|  international|    1139|
|            1|         frozen|    4007|
|           11|  personal care|    6563|
|            9|dry goods pasta|    1858|
|           13|         pantry|    5371|
|           19|         snacks|    6264|
|           20|           deli|    1322|
|            7|      beverages|    4365|
|            4|        produce|    1684|
|           15|   canned goods|    2092|
|           12|   meat seafood|     907|
|           18|         babies|    1081|
|            2|          other|     548|
|            8|           pets|     972|
|            5|        alcohol|    1054|
|           10|           bulk|      38|
+-------------+---------------+--------+
only showing top

In [8]:
query = """
select 
  count(opp.order_id) as orders, 
  p.product_name as popular_product
from order_products_prior opp, products p
where p.product_id = opp.product_id 
group by popular_product 
order by orders desc 
limit 10
"""

result_popular_products = spark.sql(query)
result_popular_products.show()

+------+--------------------+
|orders|     popular_product|
+------+--------------------+
|472565|              Banana|
|379450|Bag of Organic Ba...|
|264683|Organic Strawberries|
|241921|Organic Baby Spinach|
|213584|Organic Hass Avocado|
|176815|     Organic Avocado|
|152657|         Large Lemon|
|142951|        Strawberries|
|140627|               Limes|
|137905|  Organic Whole Milk|
+------+--------------------+



In [9]:
query = """
select
  d.department,
  count(distinct p.product_id) as products

from products p
inner join departments d
on 
d.department_id = p.department_id

group by d.department
order by products desc
limit 10
"""

result_products = spark.sql(query)
result_products.show()

+---------------+--------+
|     department|products|
+---------------+--------+
|  personal care|    6563|
|         snacks|    6264|
|         pantry|    5371|
|      beverages|    4365|
|         frozen|    4007|
|     dairy eggs|    3449|
|      household|    3084|
|   canned goods|    2092|
|dry goods pasta|    1858|
|        produce|    1684|
+---------------+--------+



## Organize and View Shopping Basket

In [10]:
# Organize the data by shopping basket
from pyspark.sql.functions import collect_set, col, count
rawData = spark.sql("select p.product_name, o.order_id from products p inner join order_products_train o where o.product_id = p.product_id")
baskets = rawData.groupBy('order_id').agg(collect_set('product_name').alias('items'))
baskets.createOrReplaceTempView('baskets')

In [11]:
baskets.show()

+--------+--------------------+
|order_id|               items|
+--------+--------------------+
|    1139|[Cinnamon Rolls w...|
|    1143|[Water, Natural P...|
|    1342|[Raw Shrimp, Seed...|
|    1591|[Cracked Wheat, S...|
|    1721|[Organic Thompson...|
|    2711|[Granny Smith App...|
|    3179|[Caramel Almond a...|
|    4092|[Organic Baby Spi...|
|    4383|[Organic Orange S...|
|    4519|[Beet Apple Carro...|
|    4935|             [Vodka]|
|    5117|[Major Dickason's...|
|    5433|[Dairy Free Unswe...|
|    6357|[Globe Eggplant, ...|
|    6552|[Plain Almond Mil...|
|    6773|[Unrefined Virgin...|
|    7168|[Dry Rubbed Thick...|
|    8231|[100% Liquid Egg ...|
|    8932|[100% Lactose Fre...|
|    9206|[Organic Large Gr...|
+--------+--------------------+
only showing top 20 rows



# Train ML Model 

To understand the frequency of items are associated with each other (e.g. peanut butter and jelly), we will use association rule mining for market basket analysis.  [Spark MLlib](http://spark.apache.org/docs/latest/mllib-guide.html) implements two algorithms related to frequency pattern mining (FPM): [FP-growth](https://spark.apache.org/docs/latest/mllib-frequent-pattern-mining.html#fp-growth) and [PrefixSpan](https://spark.apache.org/docs/latest/mllib-frequent-pattern-mining.html#prefixspan). The distinction is that FP-growth does not use order information in the itemsets, if any, while PrefixSpan is designed for sequential pattern mining where the itemsets are ordered. We will use FP-growth as the order information is not important for this use case.

> Note, we will be using the `Scala API` so we can configure `setMinConfidence`.

In [ ]:
%scala
import org.apache.spark.ml.fpm.FPGrowth

// Extract out the items 
val baskets_ds = spark.sql("select items from baskets").as[Array[String]].toDF("items")

// Use FPGrowth
val fpgrowth = new FPGrowth().setItemsCol("items").setMinSupport(0.001).setMinConfidence(0)
val model = fpgrowth.fit(baskets_ds)

import org.apache.spark.ml.fpm.FPGrowth
baskets_ds: org.apache.spark.sql.DataFrame = [items: array<string>]
fpgrowth: org.apache.spark.ml.fpm.FPGrowth = fpgrowth_9f544eac8d5b
model: org.apache.spark.ml.fpm.FPGrowthModel = FPGrowthModel: uid=fpgrowth_9f544eac8d5b, numTrainingRecords=131209

In [ ]:
%scala
// Display frequent itemsets
val mostPopularItemInABasket = model.freqItemsets
mostPopularItemInABasket.createOrReplaceTempView("mostPopularItemInABasket")

mostPopularItemInABasket: org.apache.spark.sql.DataFrame = [items: array<string>, freq: bigint]

In [ ]:
%sql
select items, freq from mostPopularItemInABasket where size(items) > 2 order by freq desc limit 20

items,freq
"List(Organic Hass Avocado, Organic Strawberries, Bag of Organic Bananas)",710
"List(Organic Raspberries, Organic Strawberries, Bag of Organic Bananas)",649
"List(Organic Baby Spinach, Organic Strawberries, Bag of Organic Bananas)",587
"List(Organic Raspberries, Organic Hass Avocado, Bag of Organic Bananas)",531
"List(Organic Hass Avocado, Organic Baby Spinach, Bag of Organic Bananas)",497
"List(Organic Avocado, Organic Baby Spinach, Banana)",484
"List(Organic Avocado, Large Lemon, Banana)",477
"List(Limes, Large Lemon, Banana)",452
"List(Organic Cucumber, Organic Strawberries, Bag of Organic Bananas)",424
"List(Limes, Organic Avocado, Large Lemon)",389


# Review Association Rules

In addition to `freqItemSets`, the `FP-growth` model also generates `association rules`.  For example, if a shopper purchases *peanut butter* , what is the likelihood that they will also purchase *jelly*.  For more information, a good reference is Susan Li's [A Gentle Introduction on Market Basket Analysis — Association Rules](https://towardsdatascience.com/a-gentle-introduction-on-market-basket-analysis-association-rules-fa4b986a40ce)

In [ ]:
%scala
// Display generated association rules.
val ifThen = model.associationRules
ifThen.createOrReplaceTempView("ifThen")

ifThen: org.apache.spark.sql.DataFrame = [antecedent: array<string>, consequent: array<string> ... 3 more fields]

In [ ]:
%sql
select antecedent as `antecedent (if)`, consequent as `consequent (then)`, confidence from ifThen order by confidence desc limit 20

antecedent (if),consequent (then),confidence
"List(Organic Raspberries, Organic Hass Avocado, Organic Strawberries)",List(Bag of Organic Bananas),0.5984251968503937
"List(Organic Cucumber, Organic Hass Avocado, Organic Strawberries)",List(Bag of Organic Bananas),0.546875
"List(Organic Kiwi, Organic Hass Avocado)",List(Bag of Organic Bananas),0.5459770114942529
"List(Organic Navel Orange, Organic Raspberries)",List(Bag of Organic Bananas),0.5412186379928315
"List(Yellow Onions, Strawberries)",List(Banana),0.5357142857142857
"List(Organic Whole String Cheese, Organic Hass Avocado)",List(Bag of Organic Bananas),0.5314685314685315
"List(Organic Navel Orange, Organic Hass Avocado)",List(Bag of Organic Bananas),0.5283018867924528
"List(Organic Raspberries, Organic Hass Avocado)",List(Bag of Organic Bananas),0.521099116781158
"List(Organic D'Anjou Pears, Organic Hass Avocado)",List(Bag of Organic Bananas),0.5170454545454546
"List(Organic Unsweetened Almond Milk, Organic Hass Avocado)",List(Bag of Organic Bananas),0.5141065830721003
